# Support Vector Machines

* Here we implement Multiclass MNIST digit recognition classifier with SVMs

In [ ]:
# Import libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split


from keras.datasets import mnist
from sklearn.svm import SVC 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import plot_roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedShuffleSplit

## Loading MNIST dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
print('Shape of training features: ', X_train.shape)
print('Shape of training labels: ', y_train.shape)
print('Shape of test features: ', X_test.shape)
print('Shape of test labels: ', y_train.shape)


In [ ]:
fig = plt.figure(figsize=(10,7))
for i in range(15):
    ax = fig.add_subplot(3, 5, i+1)
    ax.imshow(X_train[i], cmap=plt.get_cmap('gray'))
    ax.set_title('Label (y): {y}'.format(y=y_train[i]))
    plt.axis('off')

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28*28)
X_test = X_test.reshape(X_test.shape[0], 28*28)

#Normalising
X_train = X_train/255
X_test = X_test/255

Let's consider the forst 10,000 images in the training dataset and first 2000 images in the testing dataset

In [ ]:
X_train = X_train[0:10000, :]
y_train = y_train[0:10000]

X_test = X_test[0:2000, :]
y_test = y_test[0:2000]

## Linear SVM for MNIST multiclass classification

In [ ]:
pipe_1 = Pipeline([('scaler', MinMaxScaler()),
                    ('classifier', SVC(kernel='linear', C=1))])

pipe_1.fit(X_train, y_train)

acc = cross_val_score(pipe_1, X_train, y_train, cv=2)
print('Training accuracy: {:.2f} %'.format(acc.mean()*100))

In [ ]:
y_pred = pipe_1.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion Matrix')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

## Nonlinear SVM for MNIST multiclass classification

In [ ]:
pipe_2 = Pipeline([('scaler', MinMaxScaler()),
                    ('classifier', SVC(kernel='rbf', gamma =0.1, C=1))])

pipe_2.fit(X_train, y_train)

acc = cross_val_score(pipe_2, X_train, y_train, cv=2)
print('Training accuracy: {:.2f} %'.format(acc.mean()*100))

In [ ]:
y_pred = pipe_2.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion Matrix')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

### Using GridSearchCV for HPT 

In [ ]:
scaler = StandardScaler()

X_train =  scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma = gamma_range, C = C_range)
cv = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)
grid = GridSearchCV(SVC(kernel = 'rbf'), param_grid=param_grid, cv=cv)
grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.2f" %(grid.best_params_, grid.best_score_))

In [ ]:
y_pred = grid.best_estimator_.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion Matrix')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))